<a href="https://colab.research.google.com/github/priyasinghsophia/Analytics-Portfolio/blob/master/colab_retrieve_Reviews_GCP_Loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stitching & Upload tool

This file works as a tool to copy the contents of the Data Lake relevant to ingredient analysis and stitch them together in the appropriate files. It then uploads the files back to a Google Cloud Bucket to enable analysis by the team.

In [76]:
!pip install PyDrive

In [77]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [78]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

## Preparation

In [83]:
#Import the necessary libraries
from csv import reader, writer
import pandas as pd

#Authenticate to Google Drive
from google.colab import drive
drive.mount('/gdrive') 

Mounted at /gdrive


In [84]:
description_data = pd.read_csv("/gdrive/My Drive/Scrappy data/alldescriptions.csv",sep="¡")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


## Reading and Appending

###Now, explore the descriptions

In [85]:
description_data.shape

(8449, 22)

In [86]:
#See the first rows of reviews
descirption_data.head()

,ASIN,Domain,ExcludeTerm,ExclusionInProduct,IngredientInProduct,Keyword,KeywordDept,MatchTerm,ProductDescription,ProductDirections,ProductFlavor,ProductIngredientList,ProductLegal,ProductLink,ProductName,TotalCustomerRatings,ProductSafety,ProductStar,ProductSummary,RetrievedTime,ReviewsLink,UPC
0,B00384ABT0,amazon.com,"skin & hair oil, lotion",0,0,borage+oil,0,"borage oil, borage",Lean body meal replacement shakes are now avai...,Before Workouts- Take Lean Body® 1-1/2 hours p...,Chocolate,NaN,PLEASE NOTE: Product image is representative o...,/dp/B00384ABT0,LABRADA Nutrition – Lean Body High Protein Mea...,1256,Allergen Information: Contains Milk and Soy.Th...,4.4,LEAN BODY contains 35g of fast-release and slo...,2020-06-19,/LABRADA-Nutrition-Protein-Replacement-Chocola...,767644293846 885259937563 710779112742
1,B008X8MRE8,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage","General Herb Information - Borage - Bee Bread,...",NaN,Flavor_absent,Borage Oil,Statements regarding dietary supplements have ...,/dp/B008X8MRE8,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,NaN,5.0,"General Herb Information - Borage - Bee Bread,...",2020-06-19,/Borage-Starflower-Oil-Pressed-428360/product-...,064434283606
2,B004GWCKKG,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Borage oil 300mg 60 capsuleproduct description...,NaN,Flavor_absent,"Gelatin (Porcine), Glycerin And Water",Actual product packaging and materials may con...,/dp/B004GWCKKG,"Health From The Sun, Borage Oil 300 Mg, 60 Cap...",1,NaN,4.0,Health from the sun,2020-06-19,/product-reviews/B004GWCKKG,010043050511
3,B07K4461NP,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Borage Herb is also known as starflower or bee...,NaN,Flavor_absent,100% Borage Herb - Borago Officinalis L,Statements regarding dietary supplements have ...,/dp/B07K4461NP,Herba Organica - Borage Herb - Borago Officina...,2,NaN,5.0,100% Natural product without any additives\n\n...,2020-06-19,/product-reviews/B07K4461NP,No_UPC
4,B008X8S594,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",TerraVita Omega 3-6-9 EFA Blend is a balanced ...,NaN,Flavor_absent,"Gelatin, Glycerin, Water, Purified, Borage Oil...",Statements regarding dietary supplements have ...,/dp/B008X8S594,"Omega 3-6-9 EFA Blend - 1200 mg (100 softgels,...",0,NaN,0.0,TerraVita Omega 3-6-9 EFA Blend is a balanced ...,2020-06-19,/product-reviews/B008X8S594,064434279975


In [87]:
candidates_data = pd.read_csv("/gdrive/My Drive/Scrappy data/check_candidates.csv",sep="¡")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [88]:
complete_data = pd.read_csv("/gdrive/My Drive/Scrappy data/complete.csv",sep="¡")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [89]:
reviews_data = pd.read_csv("/gdrive/My Drive/Scrappy data/valid_reviews.csv",sep="¡")



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [74]:
#Let's handle repeated reviews
descirption_data.drop_duplicates(inplace=True)

In [75]:
descirption_data.shape

(8316, 22)

In [91]:
descirption_data.shape

(8316, 22)

In this case, rows will be repeated because an item can contain multiple ingredients, so an item is bound to appear in multiple searches. We will leave this be.

In [93]:
descirption_data.columns

Index(['ASIN', 'Domain', 'ExcludeTerm', 'ExclusionInProduct',
       'IngredientInProduct', 'Keyword', 'KeywordDept', 'MatchTerm',
       'ProductDescription', 'ProductDirections', 'ProductFlavor',
       'ProductIngredientList', 'ProductLegal', 'ProductLink', 'ProductName',
       'TotalCustomerRatings', 'ProductSafety', 'ProductStar',
       'ProductSummary', 'RetrievedTime', 'ReviewsLink', 'UPC'],
      dtype='object')

In [94]:
#Let's rename the column that shows the number of Customer ratings
descirption_data.rename(columns={"ProductRatings": "TotalCustomerRatings"}, inplace=True)

In [ ]:
df_descriptions.columns

Index(['ASIN', 'Domain', 'ExcludeTerm', 'ExclusionInProduct',
       'IngredientInProduct', 'Keyword', 'KeywordDept', 'MatchTerm',
       'ProductDescription', 'ProductDirections', 'ProductFlavor',
       'ProductIngredientList', 'ProductLegal', 'ProductLink', 'ProductName',
       'TotalCustomerRatings', 'ProductSafety', 'ProductStar',
       'ProductSummary', 'RetrievedTime', 'ReviewsLink', 'UPC'],
      dtype='object')

In [ ]:
df_reviews.columns

Index(['ASIN', 'ProductFlavor', 'ProductName', 'ProductNumReviews',
       'RetrievedTime', 'ReviewContent', 'ReviewDomain', 'ReviewEarly',
       'ReviewHelpful', 'ReviewId', 'ReviewLink', 'ReviewStar', 'ReviewTime',
       'ReviewVerifiedP', 'ReviewVine', 'ReviewersId', 'ReviewersName'],
      dtype='object')

In [ ]:
df_descriptions.dtypes

ASIN                     object
Domain                   object
ExcludeTerm              object
ExclusionInProduct       object
IngredientInProduct      object
Keyword                  object
KeywordDept              object
MatchTerm                object
ProductDescription       object
ProductDirections        object
ProductFlavor            object
ProductIngredientList    object
ProductLegal             object
ProductLink              object
ProductName              object
TotalCustomerRatings     object
ProductSafety            object
ProductStar              object
ProductSummary           object
RetrievedTime            object
ReviewsLink              object
UPC                      object
dtype: object

In [ ]:
# Let us convert column types to account for numbers
# and dates
df_descriptions['ExclusionInProduct'] = df_descriptions['ExclusionInProduct'].astype('int')
df_descriptions['IngredientInProduct'] = df_descriptions['IngredientInProduct'].astype('int')
df_descriptions['KeywordDept'] = df_descriptions['KeywordDept'].astype('int')
df_descriptions['TotalCustomerRatings'] = df_descriptions['TotalCustomerRatings'].apply(lambda x: x.replace(',','')).astype('int')
df_descriptions['ProductStar'] = df_descriptions['ProductStar'].astype('float')
df_descriptions['RetrievedTime'] = pd.to_datetime(df_descriptions['RetrievedTime'])
df_descriptions.dtypes

ASIN                             object
Domain                           object
ExcludeTerm                      object
ExclusionInProduct                int64
IngredientInProduct               int64
Keyword                          object
KeywordDept                       int64
MatchTerm                        object
ProductDescription               object
ProductDirections                object
ProductFlavor                    object
ProductIngredientList            object
ProductLegal                     object
ProductLink                      object
ProductName                      object
TotalCustomerRatings              int64
ProductSafety                    object
ProductStar                     float64
ProductSummary                   object
RetrievedTime            datetime64[ns]
ReviewsLink                      object
UPC                              object
dtype: object

In [ ]:
df_descriptions.head()

,ASIN,Domain,ExcludeTerm,ExclusionInProduct,IngredientInProduct,Keyword,KeywordDept,MatchTerm,ProductDescription,ProductDirections,ProductFlavor,ProductIngredientList,ProductLegal,ProductLink,ProductName,TotalCustomerRatings,ProductSafety,ProductStar,ProductSummary,RetrievedTime,ReviewsLink,UPC
0,B00384ABT0,amazon.com,"skin & hair oil, lotion",0,0,borage+oil,0,"borage oil, borage",Lean body meal replacement shakes are now avai...,Before Workouts- Take Lean Body® 1-1/2 hours p...,Chocolate,,PLEASE NOTE: Product image is representative o...,/dp/B00384ABT0,LABRADA Nutrition – Lean Body High Protein Mea...,1256,Allergen Information: Contains Milk and Soy.Th...,4.4,LEAN BODY contains 35g of fast-release and slo...,2020-06-19,/LABRADA-Nutrition-Protein-Replacement-Chocola...,767644293846 885259937563 710779112742
1,B008X8MRE8,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage","General Herb Information - Borage - Bee Bread,...",,Flavor_absent,Borage Oil,Statements regarding dietary supplements have ...,/dp/B008X8MRE8,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,,5.0,"General Herb Information - Borage - Bee Bread,...",2020-06-19,/Borage-Starflower-Oil-Pressed-428360/product-...,064434283606
2,B004GWCKKG,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Borage oil 300mg 60 capsuleproduct description...,,Flavor_absent,"Gelatin (Porcine), Glycerin And Water",Actual product packaging and materials may con...,/dp/B004GWCKKG,"Health From The Sun, Borage Oil 300 Mg, 60 Cap...",1,,4.0,Health from the sun,2020-06-19,/product-reviews/B004GWCKKG,010043050511
3,B07K4461NP,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Borage Herb is also known as starflower or bee...,,Flavor_absent,100% Borage Herb - Borago Officinalis L,Statements regarding dietary supplements have ...,/dp/B07K4461NP,Herba Organica - Borage Herb - Borago Officina...,2,,5.0,100% Natural product without any additives\n\n...,2020-06-19,/product-reviews/B07K4461NP,No_UPC
4,B008X8S594,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",TerraVita Omega 3-6-9 EFA Blend is a balanced ...,,Flavor_absent,"Gelatin, Glycerin, Water, Purified, Borage Oil...",Statements regarding dietary supplements have ...,/dp/B008X8S594,"Omega 3-6-9 EFA Blend - 1200 mg (100 softgels,...",0,,0.0,TerraVita Omega 3-6-9 EFA Blend is a balanced ...,2020-06-19,/product-reviews/B008X8S594,064434279975


Now, let's follow the process with the reviews

In [ ]:
df_reviews.dtypes

ASIN                 object
ProductFlavor        object
ProductName          object
ProductNumReviews    object
RetrievedTime        object
ReviewContent        object
ReviewDomain         object
ReviewEarly          object
ReviewHelpful        object
ReviewId             object
ReviewLink           object
ReviewStar           object
ReviewTime           object
ReviewVerifiedP      object
ReviewVine           object
ReviewersId          object
ReviewersName        object
dtype: object

In [ ]:
# Let us convert column types to account for numbers
# and dates, just like on the descriptions DF
#Dont forget to change the date type!
df_reviews['ProductNumReviews'] = df_reviews['ProductNumReviews'].apply(lambda x: x.replace(',','')).astype('int')
df_reviews['RetrievedTime'] = pd.to_datetime(df_reviews['RetrievedTime'])
df_reviews['ReviewEarly'] = df_reviews['ReviewEarly'].astype('int')
df_reviews['ReviewHelpful'] = df_reviews['ReviewHelpful'].apply(lambda x: x.replace(',','')).astype('int')
df_reviews['ReviewStar'] = df_reviews['ReviewStar'].astype('float')
df_reviews['ReviewTime'] = pd.to_datetime(df_reviews['ReviewTime'])
df_reviews['ReviewVerifiedP'] = df_reviews['ReviewVerifiedP'].astype('int')
df_reviews['ReviewVine'] = df_reviews['ReviewVine'].astype('int')
df_reviews.dtypes

ASIN                         object
ProductFlavor                object
ProductName                  object
ProductNumReviews             int64
RetrievedTime        datetime64[ns]
ReviewContent                object
ReviewDomain                 object
ReviewEarly                   int64
ReviewHelpful                 int64
ReviewId                     object
ReviewLink                   object
ReviewStar                  float64
ReviewTime           datetime64[ns]
ReviewVerifiedP               int64
ReviewVine                    int64
ReviewersId                  object
ReviewersName                object
dtype: object

### Second, explore the Reviews

In [ ]:
df_reviews.head()

,ASIN,ProductFlavor,ProductName,ProductNumReviews,RetrievedTime,ReviewContent,ReviewDomain,ReviewEarly,ReviewHelpful,ReviewId,ReviewLink,ReviewStar,ReviewTime,ReviewVerifiedP,ReviewVine,ReviewersId,ReviewersName
0,B00CRECRDU,Flavor_absent,Life Drink 227g,2,2020-06-19,I found the ingredient list on a different sit...,amazon.com,0,2,R37291O6FKJQZ8,/gp/customer-reviews/R37291O6FKJQZ8,5.0,2019-08-11,1,0,AFBZGKOBNHJG35PRKPBV5AVN5WIQ,Amazon Customer
1,B008X8MRE8,1,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,2020-06-19,"This is exactly what I wanted, a great quality...",amazon.com,0,0,R3I0S0W4S20E0E,/gp/customer-reviews/R3I0S0W4S20E0E,5.0,2018-03-19,1,0,AFWD7WVNYUOL65PVGGGEFHS6LC2Q,Rachel Guild
2,B00CRECRDU,Flavor_absent,Life Drink 227g,2,2020-06-19,I haven't tried it yet... I'm not crazy about ...,amazon.com,0,0,R6WMLG8UGE9E2,/gp/customer-reviews/R6WMLG8UGE9E2,1.0,2016-10-08,1,0,AGMBLRKLNGP2LOCXASVF2RZO5OOA,Kathleen
3,B07X2D2NX3,Slim,Lucky Eagle 10 Rolls (2 Packages) (Slim),2,2020-06-19,I stuffed right and they dibnot burn slow. the...,amazon.com,0,0,R1OHNQBHA5H3W3,/gp/customer-reviews/R1OHNQBHA5H3W3,2.0,2020-04-09,1,0,AH2DIJK54ZIW6YX6ZO4JLKOTOC3Q,Darren
4,B07X2D2NX3,Slim,Lucky Eagle 10 Rolls (2 Packages) (Slim),2,2020-06-19,Probably my favorite to use right now,amazon.com,0,0,REH8T20KTL3IW,/gp/customer-reviews/REH8T20KTL3IW,5.0,2020-03-06,1,0,AG6WRGJRAYHT3O6AAL2W5ES4BTLQ,Asia


### Merging Information

In [ ]:
#Now, let's merge both tables into a dataframe, on the ASIN

df_amzn = pd.merge(df_reviews, df_descriptions, left_on=['ASIN','ReviewDomain'],right_on=['ASIN','Domain'],how='left',suffixes=('_reviews','_product'))

df_amzn

,ASIN,ProductFlavor_reviews,ProductName_reviews,ProductNumReviews,RetrievedTime_reviews,ReviewContent,ReviewDomain,ReviewEarly,ReviewHelpful,ReviewId,ReviewLink,ReviewStar,ReviewTime,ReviewVerifiedP,ReviewVine,ReviewersId,ReviewersName,Domain,ExcludeTerm,ExclusionInProduct,IngredientInProduct,Keyword,KeywordDept,MatchTerm,ProductDescription,ProductDirections,ProductFlavor_product,ProductIngredientList,ProductLegal,ProductLink,ProductName_product,TotalCustomerRatings,ProductSafety,ProductStar,ProductSummary,RetrievedTime_product,ReviewsLink,UPC
0,B00CRECRDU,Flavor_absent,Life Drink 227g,2,2020-06-19,I found the ingredient list on a different sit...,amazon.com,0,2,R37291O6FKJQZ8,/gp/customer-reviews/R37291O6FKJQZ8,5.0,2019-08-11,1,0,AFBZGKOBNHJG35PRKPBV5AVN5WIQ,Amazon Customer,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Terranova's Life Drink provides an ideal super...,,Flavor_absent,,Statements regarding dietary supplements have ...,/dp/B00CRECRDU,Life Drink 227g,68,,4.6,,2020-06-19,/Terra-Nova-5060203792008-Life-Drink/product-r...,797142748722
1,B008X8MRE8,1,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,2020-06-19,"This is exactly what I wanted, a great quality...",amazon.com,0,0,R3I0S0W4S20E0E,/gp/customer-reviews/R3I0S0W4S20E0E,5.0,2018-03-19,1,0,AFWD7WVNYUOL65PVGGGEFHS6LC2Q,Rachel Guild,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage","General Herb Information - Borage - Bee Bread,...",,Flavor_absent,Borage Oil,Statements regarding dietary supplements have ...,/dp/B008X8MRE8,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,,5.0,"General Herb Information - Borage - Bee Bread,...",2020-06-19,/Borage-Starflower-Oil-Pressed-428360/product-...,064434283606
2,B008X8MRE8,1,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,2020-06-19,"This is exactly what I wanted, a great quality...",amazon.com,0,0,R3I0S0W4S20E0E,/gp/customer-reviews/R3I0S0W4S20E0E,5.0,2018-03-19,1,0,AFWD7WVNYUOL65PVGGGEFHS6LC2Q,Rachel Guild,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,1,"borage oil, borage","General Herb Information - Borage - Bee Bread,...",,Flavor_absent,Borage Oil,Statements regarding dietary supplements have ...,/dp/B008X8MRE8,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,,5.0,"General Herb Information - Borage - Bee Bread,...",2020-06-19,/Borage-Starflower-Oil-Pressed-428360/product-...,064434283606
3,B00CRECRDU,Flavor_absent,Life Drink 227g,2,2020-06-19,I haven't tried it yet... I'm not crazy about ...,amazon.com,0,0,R6WMLG8UGE9E2,/gp/customer-reviews/R6WMLG8UGE9E2,1.0,2016-10-08,1,0,AGMBLRKLNGP2LOCXASVF2RZO5OOA,Kathleen,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Terranova's Life Drink provides an ideal super...,,Flavor_absent,,Statements regarding dietary supplements have ...,/dp/B00CRECRDU,Life Drink 227g,68,,4.6,,2020-06-19,/Terra-Nova-5060203792008-Life-Drink/product-r...,797142748722
4,B07X2D2NX3,Slim,Lucky Eagle 10 Rolls (2 Packages) (Slim),2,2020-06-19,I stuffed right and they dibnot burn slow. the...,amazon.com,0,0,R1OHNQBHA5H3W3,/gp/customer-reviews/R1OHNQBHA5H3W3,2.0,2020-04-09,1,0,AH2DIJK54ZIW6YX6ZO4JLKOTOC3Q,Darren,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Enjoy hand-rolled natural leaves for those loo...,,Flavor_absent,,Statements regarding dietary supplements have ...,/dp/B07X2D2NX3,Lucky Eagle 10 Rolls (2 Packages) (Slim),6,,3.8,Convenient | Can't roll? No problem. These com...,2020-06-19,/Lucky-Eagle-Rolls-Packages-Slim/product-revie...,796603668180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371159,B002WHUAXG,Flavor_absent,A.Vogel Hyperiforce St John'S Wort Tablets,1,2020-06-15,Great product for those pms bues daus.,amazon.com,0,0,RXQKMVKOADVN9,/gp/customer-reviews/RXQKMVKOADVN9,5.0,2018-07-14,1,0,AGRSBWYWTU4KDY45EWPHSV3BBNYA,Naaldekoker,amazon.com,"skin & hair o

In [ ]:
df_amzn.shape

(371164, 38)

In [ ]:
# We only want to keep those rows where the 
# Item corresponds to the review and the product.
# We'll also keep those rows where the flavor wasn't specified
# For the review, but that were a verified purchase
# We also learnt that the packed size might be considered
# As a flavor, so we will take all reviews when a product
# Doesn't specify a flavor, but the reviews do. Also,
# The scrapy code will be fixed.

df_merged = df_amzn[(df_amzn['ProductFlavor_reviews']==df_amzn['ProductFlavor_product']) | ((df_amzn['ProductFlavor_reviews']=='Flavor_absent') & (df_amzn['ReviewVerifiedP']==1)) | (df_amzn['ProductFlavor_product']=='Flavor_absent')]


In [ ]:
df_merged.shape

(324876, 38)

In [ ]:
df_merged.columns

Index(['ASIN', 'ProductFlavor_reviews', 'ProductName_reviews',
       'ProductNumReviews', 'RetrievedTime_reviews', 'ReviewContent',
       'ReviewDomain', 'ReviewEarly', 'ReviewHelpful', 'ReviewId',
       'ReviewLink', 'ReviewStar', 'ReviewTime', 'ReviewVerifiedP',
       'ReviewVine', 'ReviewersId', 'ReviewersName', 'Domain', 'ExcludeTerm',
       'ExclusionInProduct', 'IngredientInProduct', 'Keyword', 'KeywordDept',
       'MatchTerm', 'ProductDescription', 'ProductDirections',
       'ProductFlavor_product', 'ProductIngredientList', 'ProductLegal',
       'ProductLink', 'ProductName_product', 'TotalCustomerRatings',
       'ProductSafety', 'ProductStar', 'ProductSummary',
       'RetrievedTime_product', 'ReviewsLink', 'UPC'],
      dtype='object')

In [ ]:
#Let's keep only the columns we will anallyze at this moment

df_valid_reviews = df_merged[['ASIN', 'ReviewId', 'Keyword', 'KeywordDept','ProductFlavor_reviews', 
       'ProductName_reviews', 'ProductNumReviews', 'ReviewContent', 
       'ReviewEarly', 'ReviewHelpful', 'ReviewStar', 'ReviewTime', 
       'ReviewVerifiedP', 'ReviewVine', 'ReviewersId', 'ReviewersName',
       'ProductIngredientList', 'TotalCustomerRatings', 'ProductStar']]

In [ ]:
df_valid_reviews.shape

(324876, 19)

## Save data to a GCS bucket / GDrive / BigQuery

In [ ]:
#Convert the dataframes into csvs
df_descriptions.to_csv('alldescriptions.csv', index=False, sep = '¡')
df_reviews.to_csv('allreviews.csv', index=False, sep = '¡')
df_merged.to_csv('complete.csv', index=False, sep = '¡')
df_valid_reviews.to_csv('valid_reviews.csv', index=False, sep = '¡')

In [ ]:
#Give variable names to all files to handle in command line
d1 = 'alldescriptions.csv'
d2 = 'allreviews.csv'
d3 = 'complete.csv'
d4 = 'valid_reviews.csv'

In [ ]:
#You can save all the files to the Google Drive if you want
!cp $d1 "/gdrive/My Drive/20200608-Scrapy_Data/Analysis"
!cp $d2 "/gdrive/My Drive/20200608-Scrapy_Data/Analysis"
!cp $d3 "/gdrive/My Drive/20200608-Scrapy_Data/Analysis"
!cp $d4 "/gdrive/My Drive/20200608-Scrapy_Data/Analysis"

In [ ]:
#No need to ulpoad. If you want, just download the csv files.
#They are available on the left pane.

In [ ]:
#If you'd rather upload to GCS, run this cell, uncommenting the lines
#Make sure you have permissions
!gsutil -m cp -r /content/$d1 gs://medfoods_reviews_bucket/
!gsutil -m cp -r /content/$d2 gs://medfoods_reviews_bucket/
!gsutil -m cp -r /content/$d3 gs://medfoods_reviews_bucket/
!gsutil -m cp -r /content/$d4 gs://medfoods_reviews_bucket/

Copying file:///content/alldescriptions.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/20.6 MiB.                                     
Copying file:///content/allreviews.csv [Content-Type=text/csv]...
\ [1/1 files][113.8 MiB/113.8 MiB] 100% Done                                    
Operation completed over 1 objects/113.8 MiB.                                    
Copying file:///content/complete.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a

As a learning exercise, the chunk below turns the dataframe containing the valid reviews into a Big Query table where you can run SQL statements.

You can connect to the bucket and download the csv, or just retrieve it from the Google Drive.

You can normalize the complete data table and then upload it to BQ to build the final version of the Data Warehouse as a learning exercise, too.

In [ ]:
#Now, run this cell to upload the valid reviews table to BigQuery
#Careful, you might end up replacing everyone's information

import pandas_gbq

df_valid_reviews.to_gbq('20200630Data.ValidReviews', 
                 project_id='medfoods', 
                 if_exists='replace',
                 progress_bar=True)


1it [01:02, 62.90s/it]


## Sanity Check

Now, just to make sure the scraper worked, let's see something: Hoe many products with reviews don't have any valid ones due to the flavor or style of the product?

In [ ]:
#Let's make a list of the ASINs that appear on the reviews
valid_asins = set(df_valid_reviews['ASIN'])

In [ ]:
#Now, let's make a list of the ASINS that have reviews
#As you recall, this was in the df_amzn table

reviewed_asins = set(df_amzn['ASIN'])

In [ ]:
#Let's create a list of the disjoint elements
missing = list(reviewed_asins - valid_asins)
df_missing = pd.DataFrame(missing,columns=['ASIN'])

In [ ]:
#Let's gather more information about those elements

df_candidates = pd.merge(df_amzn, df_missing, left_on='ASIN' ,right_on='ASIN',how='inner',suffixes=('_reviews','_product'))

In [ ]:
df_candidates.to_csv('check_candidates.csv', index=False, sep = '¡')

In [ ]:
#Save to return later
d5 = 'check_candidates.csv'
!cp $d5 "/gdrive/My Drive/20200608-Scrapy_Data/Analysis"

In [ ]:
#Now, let's understand a bit better.
df_candidates

,ASIN,ProductFlavor_reviews,ProductName_reviews,ProductNumReviews,RetrievedTime_reviews,ReviewContent,ReviewDomain,ReviewEarly,ReviewHelpful,ReviewLink,ReviewStar,ReviewTime,ReviewVerifiedP,ReviewVine,ReviewersId,ReviewersName,Domain,ExcludeTerm,ExclusionInProduct,IngredientInProduct,Keyword,KeywordDept,MatchTerm,ProductDescription,ProductDirections,ProductFlavor_product,ProductIngredientList,ProductLegal,ProductLink,ProductName_product,TotalCustomerRatings,ProductSafety,ProductStar,ProductSummary,RetrievedTime_product,ReviewsLink,UPC
0,B008X8MRE8,1,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,2020-06-19,"This is exactly what I wanted, a great quality...",amazon.com,0,0,/gp/customer-reviews/R3I0S0W4S20E0E,5.0,2018-03-19,1,0,AFWD7WVNYUOL65PVGGGEFHS6LC2Q,Rachel Guild,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage","General Herb Information - Borage - Bee Bread,...",,Flavor_absent,Borage Oil,Statements regarding dietary supplements have ...,/dp/B008X8MRE8,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,,5.0,"General Herb Information - Borage - Bee Bread,...",2020-06-19,/Borage-Starflower-Oil-Pressed-428360/product-...,064434283606
1,B008X8MRE8,1,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,2020-06-19,"This is exactly what I wanted, a great quality...",amazon.com,0,0,/gp/customer-reviews/R3I0S0W4S20E0E,5.0,2018-03-19,1,0,AFWD7WVNYUOL65PVGGGEFHS6LC2Q,Rachel Guild,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,1,"borage oil, borage","General Herb Information - Borage - Bee Bread,...",,Flavor_absent,Borage Oil,Statements regarding dietary supplements have ...,/dp/B008X8MRE8,"Borage (Starflower) Oil - 100% Pure, Cold Pres...",1,,5.0,"General Herb Information - Borage - Bee Bread,...",2020-06-19,/Borage-Starflower-Oil-Pressed-428360/product-...,064434283606
2,B07X2D2NX3,Slim,Lucky Eagle 10 Rolls (2 Packages) (Slim),2,2020-06-19,I stuffed right and they dibnot burn slow. the...,amazon.com,0,0,/gp/customer-reviews/R1OHNQBHA5H3W3,2.0,2020-04-09,1,0,AH2DIJK54ZIW6YX6ZO4JLKOTOC3Q,Darren,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Enjoy hand-rolled natural leaves for those loo...,,Flavor_absent,,Statements regarding dietary supplements have ...,/dp/B07X2D2NX3,Lucky Eagle 10 Rolls (2 Packages) (Slim),6,,3.8,Convenient | Can't roll? No problem. These com...,2020-06-19,/Lucky-Eagle-Rolls-Packages-Slim/product-revie...,796603668180
3,B07X2D2NX3,Slim,Lucky Eagle 10 Rolls (2 Packages) (Slim),2,2020-06-19,Probably my favorite to use right now,amazon.com,0,0,/gp/customer-reviews/REH8T20KTL3IW,5.0,2020-03-06,1,0,AG6WRGJRAYHT3O6AAL2W5ES4BTLQ,Asia,amazon.com,"skin & hair oil, lotion",0,1,borage+oil,0,"borage oil, borage",Enjoy hand-rolled natural leaves for those loo...,,Flavor_absent,,Statements regarding dietary supplements have ...,/dp/B07X2D2NX3,Lucky Eagle 10 Rolls (2 Packages) (Slim),6,,3.8,Convenient | Can't roll? No problem. These com...,2020-06-19,/Lucky-Eagle-Rolls-Packages-Slim/product-revie...,796603668180
4,B008X8PAXI,1,"Borage Leaf Tea (25 Tea Bags, ZIN: 427560)",2,2020-06-19,"Too weak, but good taste.",amazon.com,0,0,/gp/customer-reviews/R8AWWVFS9ANWV,1.0,2017-01-31,1,0,AGLNFXTFI3ICLX4O27Z3NM4M6FNA,Joanne Friedman,amazon.com,"skin & hair oil, lotion",0,1,borage+tea,0,"borage, borage",BORAGE ( Borago officinalis ) is a garden herb...,,Flavor_absent,Borage Leaf,Statements regarding dietary supplements have ...,/dp/B008X8PAXI,"Borage Leaf Tea (25 Tea Bags, ZIN: 427560)",2,,2.9,BORAGE ( Borago officinalis ) is a garden herb...,2020-06-19,/Borage-Leaf-Tea-tea-bags/product-reviews/B008...,064434275601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22543,B071GLPK93,2 Fl Oz,"Tame The Monster Alcohol-Free Extract, Tinctur...",1,2020-06-15,Great product for balancing my mood,amazon.com,0,0,/gp/customer-reviews/R28ONPNT30FBDY,5.0,2018-06-05,1,0,AHKXW24S3GSOY4IE

Seems like the container size is getting retrieved as a flavor. Will fix that in tthe scrapy code.

For now, all products with flavor_absent will be pulled into the valid reviews. 

Let's explore further if there are any other issues.

In [ ]:
df_candidates[df_candidates['ProductFlavor_product']!='Flavor_absent']

,ASIN,ProductFlavor_reviews,ProductName_reviews,ProductNumReviews,RetrievedTime_reviews,ReviewContent,ReviewDomain,ReviewEarly,ReviewHelpful,ReviewLink,ReviewStar,ReviewTime,ReviewVerifiedP,ReviewVine,ReviewersId,ReviewersName,Domain,ExcludeTerm,ExclusionInProduct,IngredientInProduct,Keyword,KeywordDept,MatchTerm,ProductDescription,ProductDirections,ProductFlavor_product,ProductIngredientList,ProductLegal,ProductLink,ProductName_product,TotalCustomerRatings,ProductSafety,ProductStar,ProductSummary,RetrievedTime_product,ReviewsLink,UPC
1630,B0861QG2Q3,Sleep Tea,Aprika Life – Immune Support Tea with Soothing...,52,2020-06-17,"This is a calming, yummy tea made from lemon b...",amazon.com,0,3,/gp/customer-reviews/R3MNY6RZ02V5YW,4.0,2020-05-03,1,0,AFUG4CFNRHQZW7TC2LNGJM37CQDQ,Leslie,amazon.com,"skin & hair oil, no_exclusion_string",0,1,linden+tea,0,"linden, linden, tila, tilia",is our herbalist created blend of rich natural...,,Immune Tea,,Statements regarding dietary supplements have ...,/dp/B0861QG2Q3,Aprika Life – Immune Support Tea with Soothing...,54,,4.7,"BOOSTS IMMUNITY for a healthier, richer life. ...",2020-06-17,/Aprika-Life-Peppermint-Lemongrass-Antioxidant...,No_UPC
1631,B0861QG2Q3,Sleep Tea,Aprika Life – Immune Support Tea with Soothing...,52,2020-06-17,"I've only tried it twice now, but it seems to ...",amazon.com,0,0,/gp/customer-reviews/R26V6LKT0TP2KV,5.0,2020-05-21,1,0,AFFWB3NMF5SVBU546BS2RECWPI5A,P. Ferdinand,amazon.com,"skin & hair oil, no_exclusion_string",0,1,linden+tea,0,"linden, linden, tila, tilia",is our herbalist created blend of rich natural...,,Immune Tea,,Statements regarding dietary supplements have ...,/dp/B0861QG2Q3,Aprika Life – Immune Support Tea with Soothing...,54,,4.7,"BOOSTS IMMUNITY for a healthier, richer life. ...",2020-06-17,/Aprika-Life-Peppermint-Lemongrass-Antioxidant...,No_UPC
1632,B0861QG2Q3,Sleep Tea,Aprika Life – Immune Support Tea with Soothing...,52,2020-06-17,Forget about your melatonin pills! I fall asle...,amazon.com,0,2,/gp/customer-reviews/R3BBP6O7LHQD42,5.0,2020-04-23,1,0,AFYVR6FVKXVPRMS3NLMUBNNHQJWA,Anna,amazon.com,"skin & hair oil, no_exclusion_string",0,1,linden+tea,0,"linden, linden, tila, tilia",is our herbalist created blend of rich natural...,,Immune Tea,,Statements regarding dietary supplements have ...,/dp/B0861QG2Q3,Aprika Life – Immune Support Tea with Soothing...,54,,4.7,"BOOSTS IMMUNITY for a healthier, richer life. ...",2020-06-17,/Aprika-Life-Peppermint-Lemongrass-Antioxidant...,No_UPC
1633,B0861QG2Q3,Sleep Tea,Aprika Life – Immune Support Tea with Soothing...,52,2020-06-17,I like this tea. It is a subtle flavor and has...,amazon.com,0,0,/gp/customer-reviews/R2MRH39ECYFHJJ,5.0,2020-05-18,1,0,AFHRXD7LYQGD7FSNREVE5XER7SWA,Rachel,amazon.com,"skin & hair oil, no_exclusion_string",0,1,linden+tea,0,"linden, linden, tila, tilia",is our herbalist created blend of rich natural...,,Immune Tea,,Statements regarding dietary supplements have ...,/dp/B0861QG2Q3,Aprika Life – Immune Support Tea with Soothing...,54,,4.7,"BOOSTS IMMUNITY for a healthier, richer life. ...",2020-06-17,/Aprika-Life-Peppermint-Lemongrass-Antioxidant...,No_UPC
1634,B0861QG2Q3,Sleep Tea,Aprika Life – Immune Support Tea with Soothing...,52,2020-06-17,Everything about this tea is great. The smell ...,amazon.com,0,0,/gp/customer-reviews/RS05JW0A3TO4E,5.0,2020-05-19,1,0,AFUK5MEMC2D2OM5UOBZ46DUJF7OQ,Kayla,amazon.com,"skin & hair oil, no_exclusion_string",0,1,linden+tea,0,"linden, linden, tila, tilia",is our herbalist created blend of rich natural...,,Immune Tea,,Statements regarding dietary supplements have ...,/dp/B0861QG2Q3,Aprika Life – Immune Support Tea with Soothing...,54,,4.7,"BOOSTS IMMUNITY for a healthier, richer life. ...",2020-06-17,/Aprika-Life-Peppermint-Lemongrass-Antioxidant...,No_UPC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9732,B086Z1KBV5,Raw Chaga,Innature Health Lions Mane Pow

Some of the candidates look like they were rightfully discarded, but others do not. Let's see the item that has a flavor called "Sleep tea" in the reviews butt not the product.

In [ ]:
df_amzn[df_amzn['ReviewLink']=='/gp/customer-reviews/R3MNY6RZ02V5YW']

,ASIN,ProductFlavor_reviews,ProductName_reviews,ProductNumReviews,RetrievedTime_reviews,ReviewContent,ReviewDomain,ReviewEarly,ReviewHelpful,ReviewLink,ReviewStar,ReviewTime,ReviewVerifiedP,ReviewVine,ReviewersId,ReviewersName,Domain,ExcludeTerm,ExclusionInProduct,IngredientInProduct,Keyword,KeywordDept,MatchTerm,ProductDescription,ProductDirections,ProductFlavor_product,ProductIngredientList,ProductLegal,ProductLink,ProductName_product,TotalCustomerRatings,ProductSafety,ProductStar,ProductSummary,RetrievedTime_product,ReviewsLink,UPC
47805,B0861QG2Q3,Sleep Tea,Aprika Life – Immune Support Tea with Soothing...,52,2020-06-17,"This is a calming, yummy tea made from lemon b...",amazon.com,0,3,/gp/customer-reviews/R3MNY6RZ02V5YW,4.0,2020-05-03,1,0,AFUG4CFNRHQZW7TC2LNGJM37CQDQ,Leslie,amazon.com,"skin & hair oil, no_exclusion_string",0,1,linden+tea,0,"linden, linden, tila, tilia",is our herbalist created blend of rich natural...,,Immune Tea,,Statements regarding dietary supplements have ...,/dp/B0861QG2Q3,Aprika Life – Immune Support Tea with Soothing...,54,,4.7,"BOOSTS IMMUNITY for a healthier, richer life. ...",2020-06-17,/Aprika-Life-Peppermint-Lemongrass-Antioxidant...,No_UPC


In [ ]:
df_reviews[df_reviews['ReviewLink']=='/gp/customer-reviews/R3MNY6RZ02V5YW']

,ASIN,ProductFlavor,ProductName,ProductNumReviews,RetrievedTime,ReviewContent,ReviewDomain,ReviewEarly,ReviewHelpful,ReviewLink,ReviewStar,ReviewTime,ReviewVerifiedP,ReviewVine,ReviewersId,ReviewersName
ReviewId,,,,,,,,,,,,,,,,
R3MNY6RZ02V5YW,B0861QG2Q3,Sleep Tea,Aprika Life – Immune Support Tea with Soothing...,52,2020-06-17,"This is a calming, yummy tea made from lemon b...",amazon.com,0,3,/gp/customer-reviews/R3MNY6RZ02V5YW,4.0,2020-05-03,1,0,AFUG4CFNRHQZW7TC2LNGJM37CQDQ,Leslie


This behavior is odd, as there are no reviews for the "immune tea" flavor. Furthermore, neither the reviews, nor the product, reference a flavor field through the browser view, so the issue might just have been on a hidden tag, or on the locale of the volunteer retrieving this ingredient.

## Having reviewed corner cases, we'll conclude this analysis and review again when there are more files in the folder.